In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as tfrms
from torchvision import datasets
import torch.nn.functional as F


class DCNN(nn.Module):
    def __init__(self, num_classes=43):
        super(DCNN, self).__init__()

        # First Convolutional Block
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)  # Batch normalization layer
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Second Convolutional Block
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Third Convolutional Block
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fourth Convolutional Block (Additional layer for more capacity)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully Connected Layers
        # Adjust input size based on the output of the last pooling layer
        self.fc1 = nn.Linear(256 * 4 * 4, 512)  # Adjusted for new conv layers
        self.dropout = nn.Dropout(0.5)  # Dropout layer
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        # Convolutional layers with ReLU activation and BatchNorm
        x = self.pool1(F.elu(self.bn1(self.conv1(x))))
        x = self.pool2(F.elu(self.bn2(self.conv2(x))))
        x = self.pool3(F.elu(self.bn3(self.conv3(x))))
        x = self.pool4(F.elu(self.bn4(self.conv4(x))))

        # Flatten the tensor
        x = x.view(x.size(0), -1)  # Flatten for fully connected layer

        # Fully connected layers with ReLU and Dropout
        x = F.elu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Step 2: Preprocess and load data
transform = tfrms.Compose([
    tfrms.Resize((64, 64)),
    tfrms.Grayscale(num_output_channels=3),
    tfrms.RandomHorizontalFlip(1),
    tfrms.RandomVerticalFlip(1),
    tfrms.ToTensor(),
    tfrms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Example: If you had a directory for training images
train_data = datasets.ImageFolder(root='./train', transform=transform)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

# Step 3: Initialize the model, loss function, and optimizer
model = DCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

# Step 4: Train the model
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")

# Step 5: Save the model (optional)
torch.save(model.state_dict(), 'road_sign_model.pth')

# Step 6: Evaluate on test data (optional)
# Use similar DataLoader and model.eval() for validation/test

Epoch [1/10], Loss: 4.598021984100342
Epoch [2/10], Loss: 2.783665041128794
Epoch [3/10], Loss: 2.16161381204923
Epoch [4/10], Loss: 2.1884547074635825
Epoch [5/10], Loss: 2.067438095808029
Epoch [6/10], Loss: 1.9862418870131175
Epoch [7/10], Loss: 1.8109688361485798
Epoch [8/10], Loss: 1.7891670266787212
Epoch [9/10], Loss: 1.6266494393348694
Epoch [10/10], Loss: 1.6417242685953777
